In [ ]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

%config InlineBackend.figure_format = 'retina'
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])

#### Code 2.1

In [ ]:
ways = np.array([0, 3, 8, 9, 0])
ways / ways.sum()

#### Code 2.2

$$Pr(W, L \mid p) =  \frac{(W + L)!}{W!L!} p^W (1 − p)^{L}$$


The probability of observing six W’s in nine tosses—under a value of p=0.5

In [ ]:
W = 6
L = 3
n = W + L
p = 0.5

stats.binom.pmf(W, n=n, p=p)

#### Code 2.3 and 2.5

Computing the posterior using a grid approximation.

In the book the following code is not inside a function, but this way is easier to play with different parameters

In [ ]:
def posterior_grid_approx(grid_points=5, success=6, tosses=9, prior=None):
    """
    """
    # define grid
    p_grid = np.linspace(0, 1, grid_points)

    # define prior
    if prior is None:
        prior = np.repeat(5, grid_points)  # uniform
    elif prior == 'step':
        prior = (p_grid >= 0.5).astype(int)  # truncated
    elif prior == 'peaked':
        prior = np.exp(- 5 * abs(p_grid - 0.5))  # double exp

    # compute likelihood at each point in the grid
    likelihood = stats.binom.pmf(success, tosses, p_grid)

    # compute product of likelihood and prior
    unstd_posterior = likelihood * prior

    # standardize the posterior, so it sums to 1
    posterior = unstd_posterior / unstd_posterior.sum()
    return p_grid, posterior

#### Code 2.4 (uniform prior)

In [ ]:
def plot_posterior(points, success, tosses, prior=None):
    p_grid, posterior = posterior_grid_approx(points, success, tosses, prior=prior)
    plt.plot(p_grid, posterior, 'o-', label='success = {}\ntosses = {}'.format(success, tosses))
    plt.xlabel('probability of water', fontsize=14)
    plt.ylabel('posterior probability', fontsize=14)
    plt.title('{} points'.format(points))
    plt.legend(loc=0);

In [ ]:
points = 20
w, n = 6, 9
plot_posterior(points, success=w, tosses=n)

#### Code 2.5

##### Step Prior: 0 prob for values < 0.5

In [ ]:
prior = 'step'  # See posterior_grid_approx function for how this is calculated
points = 20
w, n = 6, 9
plot_posterior(points, success=w, tosses=n, prior=prior)

##### Peaked Prior: Increased likelihood at 0.5

In [ ]:
prior = 'peaked'  # See posterior_grid_approx function for how this is calculated
points = 20
w, n = 6, 9
plot_posterior(points, success=w, tosses=n, prior=prior)

#### Code 2.6

Computing the posterior using the quadratic aproximation

In [ ]:
def precis(mean_q, std_q):
    """
    Simplifies printing a summary of quadratic approximation -- kind of like the precis function in the book
    """
    print('Mean: ' + str(mean_q['p']))
    print('StdDev: ' + str(std_q))
    
    norm = stats.norm(mean_q, std_q)
    prob = .89
    z = stats.norm.ppf([(1-prob)/2, (1+prob)/2])
    pi = mean_q['p'] + std_q * z 
    print('5.5%: ' + str(pi[0]))
    print('94.5%: ' + str(pi[1]))
    
    

In [ ]:
data = np.repeat((0, 1), (3, 6))
with pm.Model() as normal_aproximation:
    p = pm.Uniform('p', 0, 1)  # Uniform prior
    w = pm.Binomial('w', n=len(data), p=p, observed=data.sum())  # binomial likelihood
    mean_q = pm.find_MAP()
    std_q = ((1/pm.find_hessian(mean_q, vars=[p]))**0.5)[0]

precis(mean_q, std_q)

#### Code 2.7

Computing the posterior using the analytical calculation & comparing to quadratic approximation

In [ ]:
# analytical calculation
W, L = 6, 3
x = np.linspace(0, 1, 100)
plt.plot(x, stats.beta.pdf(x , W+1, L+1),
         label='True posterior')

# quadratic approximation
plt.plot(x, stats.norm.pdf(x, mean_q['p'], std_q),
         label='Quadratic approximation')
plt.legend(loc=0, fontsize=13)

plt.title('n = {}'.format(W+L), fontsize=14)
plt.xlabel('Proportion water', fontsize=14)
plt.ylabel('Density', fontsize=14);

#### Code 2.8

MCMC approximation

In [ ]:
n_samples = 1000
p = np.repeat(None, n_samples)
p[0] = 0.5
W = 6
L = 3
for i in range(1,n_samples):
    p_new = stats.norm(p[i-1], 0.1).rvs(size=1)[0]
    if p_new < 0:
        p_new = math.abs(p_new)
    if p_new > 1:
        p_new = 2 - p_new
    q0 = stats.binom.pmf(W, W+L, p[i-1])
    q1 = stats.binom.pmf(W, W+L, p_new)
    if stats.uniform().rvs(size=1)[0] < q1/q0:
        p[i] = p_new
    else:
        p[i] = p[i-1]

#### Code 2.9

In [ ]:
x = np.linspace(0, 1, 100)
plt.plot(x, stats.beta.pdf(x , W+1, L+1),
          label='True posterior')

plt.hist(list(p), range=(0,1), density=True, bins=100, label="MCMC")
plt.legend(loc=0, fontsize=13)

### Book Problems

#### 2M1: Compute and plot the grix approximate posterior distribution for each of the following sets of observations. Assume a uniform prior.

1) W, W, W

In [ ]:
points = 20
w, n = 3, 3

plot_posterior(points, success=w, tosses=n, prior=None)

2) W, W, W, L

In [ ]:
points = 20
w, n = 3, 4

plot_posterior(points, success=w, tosses=n, prior=None)

3) L, W, W, L, W, W, W

In [ ]:
points = 20
w, n = 5, 7

plot_posterior(points, success=w, tosses=n, prior=None)

#### 2M2:  Compute and plot the grix approximate posterior distribution for each of the following sets of observations. Assume a prior for p that is equal to zero when p < 0.5 and is a positive constant when p ≥ 0.5

1) W, W, W

In [ ]:
points = 20
w, n = 3, 3

plot_posterior(points, success=w, tosses=n, prior='step')

2) W, W, W, L

In [ ]:
points = 20
w, n = 3, 4

plot_posterior(points, success=w, tosses=n, prior='step')

3) L, W, W, L, W, W, W

In [ ]:
points = 20
w, n = 5, 7

plot_posterior(points, success=w, tosses=n, prior='step')

#### 2M3
Suppose there are two globes, one for Earth and one for Mars. The Earth globe is 70% covered in water. The Mars globe is 100% land. Further suppose that one of these globes—you don’t know which—was tossed in the air and produced a “land” observation. Assume that each globe was equally likely to be tossed. Show that the posterior probability that the globe was the Earth, conditional on seeing “land” (Pr(Earth|land)), is 0.23.

In [ ]:
# Goal: Pr(Globe|Land) = [0.23, 0.77]
# Posterior = prob of data(likelihood) * prior / avg. prob of data
# Pr(Globe|land) = Pr(land|Globe) * Pr(Globe) / Pr(Land)

likelihood = np.array([0.3, 1])
prior = np.array([0.5, 0.5])

unstandardized_posterior = prior * likelihood
posterior = unstandardized_posterior / sum(unstandardized_posterior)
print(posterior)

#### 2M4

Suppose you have a deck with only three cards. Each card has two sides, and each side is either black or white. One card has two black sides. The second card has one black and one white side. The third card has two white sides. Now suppose all three cards are placed in a bag and shuffled. Someone reaches into the bag and pulls out a card and places it flat on a table. A black side is shown facing up, but you don’t know the color of the side facing down. Show that the probability that the other side is also black is 2/3. Use the counting method (Section 2 of the chapter) to approach this problem. This means counting up the ways that each card could produce the observed data (a black side facing up on the table).

- Card B/B: 2 ways
- Card W/B: 1 way
- Card W/W: 0 ways


- Total ways to see B: 3
- Possible ways for B/B: 2

2 / 3 = 2/3

#### 2M5

Now suppose there are four cards: B/B, B/W, W/W, and another B/B. Again suppose a card is drawn from the bag and a black side appears face up. Again calculate the probability that the other side is black.

- Card B/B: 2 ways
- Card B/B: 2 ways
- Card W/B: 1 way
- Card W/W: 0 ways


- Total ways to see B: 5
- Possible ways for B/B: 4

4 / 5 = 4/5

#### 2M6

Imagine that black ink is heavy, and so cards with black sides are heavier than cards with white sides. As a result, it’s less likely that a card with black sides is pulled from the bag. So again assume there are three cards: B/B, B/W, and W/W. After experimenting a number of times, you conclude that for every way to pull the B/B card from the bag, there are 2 ways to pull the B/W card and 3 ways to pull the W/W card. Again suppose that a card is pulled and a black side appears face up. Show that the probability the other side is black is now 0.5. Use the counting method, as before.

- Card B/B (1): 2 ways
- Card W/B (2): 1 ways
- Card W/W (3): 0 ways


- Total ways to see B: (2 * 1) + (1 * 2) + (0 * 3) = 4
- Possible ways for B/B: (2 * 1) = 2

2 / 4 = 0.5

#### 2M7

Assume again the original card problem, with a single card showing a black side face up.Before looking at the other side, we draw another card from the bag and lay it face up on the table. The face that is shown on the new card is white. Show that the probability that the first card, the one showing a black side, has black on its other side is now 0.75. Use the counting method, if you can. Hint: Treat this like the sequence of globe tosses, counting all the ways to see each observation, for each possible first card.

- B/B, W/B: 2 ways
- B/B, W/W: 4 ways
- W/B, B/B: 0 ways
- W/B, W/W: 2 ways
- W/W, B/B: 0 ways
- W/W, W/B: 0 ways


- Total possible ways to see B, W: 8
- Possible ways for B/B, W: 6
    

6/8 = 0.75

#### 2H1

Suppose there are two species of panda bear. Both are equally common in the wild and live in the same places. They look exactly alike and eat the same food, and there is yet no genetic assay capable of telling them apart. They differ however in their family sizes. Species A gives birth to twins 10% of the time, otherwise birthing a single infant. Species B births twins 20% of the time, otherwise birthing singleton infants. Assume these numbers are known with certainty, from many years of field research.

Now suppose you are managing a captive panda breeding program. You have a new female panda of unknown species, and she has just given birth to twins. What is the probability that her next birth will also be twins?

In [ ]:
# Goal: Pr(twins_1|twins_0)
# Goal: Pr(twins_1|twins_0) = Pr(twins|Species=A) * Pr(Species=A|twins_0) + Pr(twins|Species=B) * Pr(Species=B|twins_0)

# Posterior = prob of data(likelihood) * prior / avg. prob of data
# Pr(Species|twins_0) = Pr(twins|Species) * Pr(Species) / Pr(twins)

likelihood = np.array([0.1, 0.2])  # species A, species B (Pr(twins|Species))
prior = np.array([0.5, 0.5])  # (Pr(Species))

unstandardized_posterior = prior * likelihood
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|twins_0)

# Probability next birth is twins given first birth was twins
# Pr(Species|twins_1) = Pr(twins|Species) * Pr(Species|twins_0) / Pr(twins)
prior = posterior
unstandardized_posterior = prior * likelihood  # Pr(twins|Species) * Pr(Species|twins_0)
prob_twins = sum(unstandardized_posterior)  # Pr(twins|Species=A) * Pr(Species=A) + Pr(twins|Species=B) * Pr(Species=B)

print(f'Pr(twins_1|twins_0) = {prob_twins}')

#### 2H2

Recall all the facts from the problem above. Now compute the probability that the panda we have is from species A, assuming we have observed only the first birth and that it was twins.

In [ ]:
# Goal: Pr(Species|twins_0) = Pr(twins|Species) * Pr(Species) / Pr(twins)
# Posterior = prob of data(likelihood) * prior / avg. prob of data

likelihood = np.array([0.1, 0.2])  # species A, species B (Pr(twins|Species))
prior = np.array([0.5, 0.5])  # (Pr(Species))

unstandardized_posterior = prior * likelihood
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|twins_0)

print(f'Pr(Species|twins_0) = {posterior}')  # [Pr(Species=A|twins), Pr(Species=B|twins)]

#### 2H3

Continuing on from the previous problem, suppose the same panda mother has a second birth and that it is not twins, but a singleton infant. Compute the posterior probability that this panda is species A.

In [ ]:
# Goal: Pr(Species|twins_0, single_1) = Pr(twins_0, single_1|Species) * Pr(Species) / Pr(twins_0, single_1)
# Posterior = prob of data(likelihood) * prior / avg. prob of data

# Since the first and second births are independent, we can just calculate Pr(Species|twins)
# and use it as the prior for Pr(Species) when calculating Pr(single_1|Species)

likelihood_twins = np.array([0.1, 0.2])  # species A, species B (Pr(twins|Species))
prior = np.array([0.5, 0.5])  # (Pr(Species))

unstandardized_posterior = prior * likelihood_twins
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|twins_0)

# Use previous posterior as prior
prior = posterior
likelihood_single = np.array([0.9, 0.8])   # species A, species B (Pr(single|Species))

unstandardized_posterior = prior * likelihood_single
posterior = unstandardized_posterior / sum(unstandardized_posterior)
print(f'Pr(Species|twins_0, single_1) = {posterior}')  # [Pr(Species=A|twins_0, single_1), Pr(Species=B|twins_0, single_1)]

#### 2H4

A common boast of Bayesian statisticians is that Bayesian inference makes it easy to use all of the data, even if the data are of different types.
So suppose now that a veterinarian comes along who has a new genetic test that she claims can identify the species of our mother panda. But the test, like all tests, is imperfect. This is the information you have about the test:


- The probability it correctly identifies a species A panda is 0.8.
- The probability it correctly identifies a species B panda is 0.65.


The vet administers the test to your panda and tells you that the test is positive for species A. First ignore your previous information from the births and compute the posterior probability that your panda is species A. Then redo your calculation, now using the birth data as well.

In [ ]:
# Goal: Pr(Species|Test=A) = Pr(Test=A|Species) * Pr(Species) / Pr(Test=A)

# What we know:
# Pr(Test=A|Species=A) = 0.8
# Pr(Test=B|Species=B) = 0.65
# Pr(Test=A|Species=B) = 0.35
# Pr(Test=B|Species=A) = 0.2

likelihood = np.array([0.8, 0.35])  # (Pr(Test=A|Species))
prior = np.array([0.5, 0.5])  # (Pr(Species))

unstandardized_posterior = prior * likelihood
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|Test=A)
print(f'Pr(Species|Test=A) = {posterior}')  # [Pr(Species=A|Test=A), Pr(Species=B|Test=A)]

In [ ]:
# Goal: Pr(Species|Test=A, twins_0, single_1)

# We can just use the posteriors of each as the priors of the next

# First the test
likelihood_test_a = np.array([0.8, 0.35])  # (Pr(Test=A|Species))
prior = np.array([0.5, 0.5])  # (Pr(Species))

unstandardized_posterior = prior * likelihood_test_a
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|Test=A)
print(f'Pr(Species|Test=A) = {posterior}')

# Then the twins
prior = posterior
likelihood_twins = np.array([0.1, 0.2])  # species A, species B (Pr(twins|Species))

unstandardized_posterior = prior * likelihood_twins
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|Test=A, twins_0)
print(f'Pr(Species|Test=A, twins_0) = {posterior}')

# Then the single
prior = posterior
likelihood_single = np.array([0.9, 0.8])  # species A, species B (Pr(single|Species))

unstandardized_posterior = prior * likelihood_single
posterior = unstandardized_posterior / sum(unstandardized_posterior)  # Pr(Species|Test=A, twins_0, single_1)
print(f'Pr(Species|Test=A, twins_0, single_1) = {posterior}')  # [Pr(Species=A|Test=A, twins_0, single_1), Pr(Species=B|Test=A, twins_0, single_1)]

### Homework

#### 1)
Suppose the globe tossing data had turned out to be 8 water in 15 tosses. Construct the posterior distribution, using grid approximation. use the same flat prior as before.

In [ ]:
points = 30
w, n = 8, 15
prior = np.repeat(5,points)
p_grid, posterior = posterior_grid_approx(points, w, n, prior)
plt.plot(p_grid, posterior, 'o-', label='success = {}\ntosses = {}'.format(w, n))
plt.xlabel('probability of water', fontsize=14)
plt.ylabel('posterior probability', fontsize=14)
plt.title('{} points'.format(points))
plt.legend(loc=0);

#### 2)
Start over in 1, but now use a prior that is zero below p=0.5 and a constant above p=0.5. This corresponds to prior info that a majority of Earth's surface is water. What difference does the better prior make? If it helps, compare posterior distributions (using both priors) to the true value p=0.7

In [ ]:
points = 30
w, n = 8, 15
prior = (np.linspace(0,1,points) >= 0.5).astype(int)
p_grid, posterior = posterior_grid_approx(points, w, n, prior)
plt.plot(p_grid, posterior, 'o-', label='success = {}\ntosses = {}'.format(w, n))
plt.xlabel('probability of water', fontsize=14)
plt.ylabel('posterior probability', fontsize=14)
plt.title('{} points'.format(points))
plt.legend(loc=0);

The better prior (zero below p = 0.5) yields a higher posterior probability that the probability of water = 0.7. This is because the area under the curve must sum to 1, and there is less space below this posterior's curve (meaning that the graph is streched higher).

#### 3)
This problem is more open-ended than the others. Feel free to colaborate on the solution. Suppose you want to estimate the Earth's proportion of water very precisely. Specifically, you want the 99% percentile interval of the posterior distribution of p to be only 0.05 wide. This means the distance between the upper and lower bound of the interval should be 0.05. How many times will you have to toss the globe to do this? I won't require a precise answer. I'm honestly more interested in your approach.

In [ ]:
def percentile_interval(data, confidence):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n-1)
    interval = h*2
    return interval

In [ ]:
def calculate_interval_based_on_tosses(tosses):
    p_true = 0.7
    w = stats.binom.rvs(tosses,p_true)
    p_grid = np.linspace(0,1,1000)
    prior = np.repeat(1,1000)
    prob_data = stats.binom.pmf(w,tosses, p_grid)
    posterior = prob_data * prior
    posterior = posterior / sum(posterior)
    samples = np.random.choice(p_grid, size=1000, p=posterior, replace=True)
    interval_vals = np.percentile(samples, [0.05, 99.5])
    print(interval_vals)
    interval = interval_vals[1] - interval_vals[0]
    return interval
        
        
        
#     # define grid
#     p_grid = np.linspace(0, 1, grid_points)

#     # define prior
#     if prior is None:
#         prior = np.repeat(5, grid_points)  # uniform
#     #prior = (p_grid >= 0.5).astype(int)  # truncated
#     #prior = np.exp(- 5 * abs(p_grid - 0.5))  # double exp

#     # compute likelihood at each point in the grid
#     likelihood = stats.binom.pmf(success, tosses, p_grid)

#     # compute product of likelihood and prior
#     unstd_posterior = likelihood * prior

#     # standardize the posterior, so it sums to 1
#     posterior = unstd_posterior / unstd_posterior.sum()
#     return p_grid, posterior

In [ ]:
calculate_interval_based_on_tosses(2000)

In [ ]:
import sys, IPython, scipy, matplotlib, platform
print("This notebook was createad on a computer %s running %s and using:\nPython %s\nIPython %s\nPyMC3 %s\nNumPy %s\nSciPy %s\nMatplotlib %s\n" % (platform.machine(), ' '.join(platform.linux_distribution()[:2]), sys.version[:5], IPython.__version__, pm.__version__, np.__version__, scipy.__version__, matplotlib.__version__))